# Articles Summarization using Transformers and BeautifulSoup

To scrape a given article from the web using BeautifulSoup, and to summarize the article using Transformers.
<br><br>
References for this tutorial:
- https://www.youtube.com/watch?v=JctmnczWg0U
- https://github.com/nicknochnack/Longform-Summarization-with-Hugging-Face/blob/main/LongSummarization.ipynb
- https://blog.finxter.com/classification-of-star-wars-lego-images-using-cnn-and-transfer-learning/


# Install and Import Modules

Enable GPU:
- Method 1: Edit tab --> Notebook settings --> GPU --> Save.
- Method 2: Runtime tab --> Change runtime type --> GPU --> Save.

In [1]:
!pip install transformers

     |████████████████████████████████| 2.6 MB 5.1 MB/s 
     |████████████████████████████████| 3.3 MB 41.1 MB/s 
     |████████████████████████████████| 636 kB 56.3 MB/s 
     |████████████████████████████████| 895 kB 52.3 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [2]:
from transformers import pipeline
from bs4 import BeautifulSoup
import requests

# Load Model

In [4]:
summarizer = pipeline("summarization")

# Web Scraping

In [16]:
URL = "https://blog.finxter.com/classification-of-star-wars-lego-images-using-cnn-and-transfer-learning/"

headers = {‘User-Agent’: ‘Mozilla/5.0 (Windows NT 10.0;Win64) AppleWebkit/537.36 (KHTML,
like Gecko) Chrome/89.0.4389.82 Safari/537.36’}

r = requests.get(URL, headers=headers)
#r = requests.get(URL)

In [17]:
soup = BeautifulSoup(r.text, 'html.parser')
results = soup.find_all(['h1', 'p'])
text = [result.text for result in results]
ARTICLE = ' '.join(text)

In [18]:
ARTICLE

'🐍 15 Python & Computer Science Courses: Machine Learning, Data Science, Python Basics, … ☕\xa03 Freelancer Courses: Upwork, Fiverr, Freelance Developing 📹\xa0 100h++ growing library of Python video courses 🧩\xa0Unlimited Python Puzzles at Finxter.com 🌟\xa0Downloadable Python Certificates Join Premium Bitcoin is the hardest asset in the world. Read this article on why and how Finxter adopted Bitcoin as its treasury reserve asset. [Article] 3 Steps to Adopt Bitcoin as a Treasury Reserve Asset How to Classify Star Wars Lego Images using CNN and Transfer Learning This tutorial is about training deep learning (DL) models to classify Star Wars Lego images. We use the TensorFlow library to create and compare the image classifiers. Are you looking for interesting deep learning projects that are suitable for beginners? Do not worry, this is not another MNIST image classification tutorial. Instead, we are going to classify some Star Wars Lego images using the TensorFlow library. This tutorial w

# Text Processing

In [52]:
content = ARTICLE.split('How to Classify Star Wars Lego Images using CNN and Transfer Learning ')
content

['🐍 15 Python & Computer Science Courses: Machine Learning, Data Science, Python Basics, … ☕\xa03 Freelancer Courses: Upwork, Fiverr, Freelance Developing 📹\xa0 100h++ growing library of Python video courses 🧩\xa0Unlimited Python Puzzles at Finxter.com 🌟\xa0Downloadable Python Certificates Join Premium Bitcoin is the hardest asset in the world. Read this article on why and how Finxter adopted Bitcoin as its treasury reserve asset. [Article] 3 Steps to Adopt Bitcoin as a Treasury Reserve Asset ',
 'This tutorial is about training deep learning (DL) models to classify Star Wars Lego images. We use the TensorFlow library to create and compare the image classifiers. Are you looking for interesting deep learning projects that are suitable for beginners? Do not worry, this is not another MNIST image classification tutorial. Instead, we are going to classify some Star Wars Lego images using the TensorFlow library. This tutorial will sharpen your knowledge about convolutional neural networks a

In [53]:
content = content[1]
content

'This tutorial is about training deep learning (DL) models to classify Star Wars Lego images. We use the TensorFlow library to create and compare the image classifiers. Are you looking for interesting deep learning projects that are suitable for beginners? Do not worry, this is not another MNIST image classification tutorial. Instead, we are going to classify some Star Wars Lego images using the TensorFlow library. This tutorial will sharpen your knowledge about convolutional neural networks and transfer learning. Intrigued? Let’s get started. Feel free to download the script for this tutorial from this GitHub repo. We will execute it in Google Colab and use some free GPU resources for model training. If you would like to try Google Colab out, head over to the site and sign up using your Gmail account. It looks like Jupyter Notebook but with its storage location in your Google Drive. Upload the script onto your Google Colab and execute it along as you read through this article. Execute

In [54]:
content = content.replace('.', '.<eos>')
content = content.replace('?', '?<eos>')
content = content.replace('!', '!<eos>')
content

'This tutorial is about training deep learning (DL) models to classify Star Wars Lego images.<eos> We use the TensorFlow library to create and compare the image classifiers.<eos> Are you looking for interesting deep learning projects that are suitable for beginners?<eos> Do not worry, this is not another MNIST image classification tutorial.<eos> Instead, we are going to classify some Star Wars Lego images using the TensorFlow library.<eos> This tutorial will sharpen your knowledge about convolutional neural networks and transfer learning.<eos> Intrigued?<eos> Let’s get started.<eos> Feel free to download the script for this tutorial from this GitHub repo.<eos> We will execute it in Google Colab and use some free GPU resources for model training.<eos> If you would like to try Google Colab out, head over to the site and sign up using your Gmail account.<eos> It looks like Jupyter Notebook but with its storage location in your Google Drive.<eos> Upload the script onto your Google Colab an

In [56]:
sentences = content.split('<eos>')
sentences

['This tutorial is about training deep learning (DL) models to classify Star Wars Lego images.',
 ' We use the TensorFlow library to create and compare the image classifiers.',
 ' Are you looking for interesting deep learning projects that are suitable for beginners?',
 ' Do not worry, this is not another MNIST image classification tutorial.',
 ' Instead, we are going to classify some Star Wars Lego images using the TensorFlow library.',
 ' This tutorial will sharpen your knowledge about convolutional neural networks and transfer learning.',
 ' Intrigued?',
 ' Let’s get started.',
 ' Feel free to download the script for this tutorial from this GitHub repo.',
 ' We will execute it in Google Colab and use some free GPU resources for model training.',
 ' If you would like to try Google Colab out, head over to the site and sign up using your Gmail account.',
 ' It looks like Jupyter Notebook but with its storage location in your Google Drive.',
 ' Upload the script onto your Google Colab a

In [57]:
max_word_len = 500
current_block = 0 
blocks = []

# split sentences into word blocks of 500 words because that's the upper limit of transformers
for sentence in sentences:
    if len(blocks) == current_block + 1: 
        if len(blocks[current_block]) + len(sentence.split(' ')) <= max_word_len:
            blocks[current_block].extend(sentence.split(' '))
        else:
            current_block += 1
            blocks.append(sentence.split(' '))
    else:
        blocks.append(sentence.split(' '))

# join words of each block into a sentence string
for block_id in range(len(blocks)):
    blocks[block_id] = ' '.join(blocks[block_id])
  
print(f"Total number of blocks: {len(blocks)}")

Total number of blocks: 4


In [58]:
blocks[0]

'This tutorial is about training deep learning (DL) models to classify Star Wars Lego images.  We use the TensorFlow library to create and compare the image classifiers.  Are you looking for interesting deep learning projects that are suitable for beginners?  Do not worry, this is not another MNIST image classification tutorial.  Instead, we are going to classify some Star Wars Lego images using the TensorFlow library.  This tutorial will sharpen your knowledge about convolutional neural networks and transfer learning.  Intrigued?  Let’s get started.  Feel free to download the script for this tutorial from this GitHub repo.  We will execute it in Google Colab and use some free GPU resources for model training.  If you would like to try Google Colab out, head over to the site and sign up using your Gmail account.  It looks like Jupyter Notebook but with its storage location in your Google Drive.  Upload the script onto your Google Colab and execute it along as you read through this arti

In [59]:
blocks[1]

' To do that, click on the Google Drive icon at the left of the interface to mount it – as shown in Figure 1.  You will see a folder named “drive” appear on the data repository once it is mounted.  Next, we are going to restructure our data folder.  We want to create a train set, a validation set, and a test set for modeling and evaluation.  Execute the following code to restructure the data folder: In the code, we defined a base directory for the Google Drive folder.  We also reassigned names to the five data subfolders.  We defined the proportion of image distribution in each folder to be copied onto the new sets.  For example, in a subfolder of 10 images, 6 images will be copied to the train set, 3 to the validation set, and the remaining images to the test set.  The variables total_train, total_val, and total_test are counters to calculate the total number of images in the said folders.  Execute the following lines to create new folders: Executing the following code will copy the i

In [60]:
blocks[2]

' Execute the following lines to create a basic convolutional neural network model: If you want to see how the model architecture looks like, execute this line: From the printout, we see that the model consists of two convolution layers and max-pooling layers, as well as a flatten layer.  It is followed by a dense layer and an output layer of 5 units.  The unit of output layer determines the number of categories for model prediction.  Let’s compile the model with the loss, accuracy, and optimization functions: Note that there is no one-hot encoding for the labels (and no Softmax at the output layer).  The loss function is chosen based on that.  Execute the following lines for the actual model training: We defined 30 epochs and an EarlyStopping() function to the fit() function.  The EarlyStopping() function will end the model training when the criteria are fulfilled.  The fit() function trains and validates the model.  We also saved the model as an H5 file when the training is completed

In [61]:
blocks[3]

' Now we plot its losses and accuracies, as well as testing the model with the same batch of test data: As shown in Figure 5, the model got three out of four sample data right.  The training loss in Figure 4 looks better than the previous model.  Both the train and validation accuracies achieved a higher and less fluctuating outcome.  So we can say that the transfer learning model performs better than the basic CNN model.  Note that both the models can be further optimized, so do not take this code example as an end-all result.  Yay!  We learned about image classification by implementing a basic CNN and a transfer learning CNN.  I hope this was a fun learning process for you!  If you encounter any issues and would like an in-depth walkthrough of the code, the video explanation is there to help you out.  Happy learning! '

# Text Summarization

In [62]:
res = summarizer(blocks, max_length=120, min_length=30, do_sample=False)

In [63]:
# summarized text for block 1
res[0]

{'summary_text': ' This tutorial is about training deep learning (DL) models to classify Star Wars Lego images . We use the TensorFlow library to create and compare the image classifiers . Feel free to download the script for this tutorial from this GitHub repo . We will execute it in Google Colab and use free GPU resources for model training .'}

In [66]:
text = ' '.join([summ['summary_text'] for summ in res])
text

' This tutorial is about training deep learning (DL) models to classify Star Wars Lego images . We use the TensorFlow library to create and compare the image classifiers . Feel free to download the script for this tutorial from this GitHub repo . We will execute it in Google Colab and use free GPU resources for model training .  We want to create a train set, a validation set, and a test set for modeling and evaluation . In the code, we defined a base directory for the Google Drive folder . We also reassigned names to the five data subfolders . The proportion of image distribution in each folder to be copied onto the new sets .  The basic CNN model is a basic convolutional neural network model . It consists of two convolution layers and max-pooling layers, as well as a flatten layer . It is followed by a dense layer and an output layer of 5 units . Figure 2 shows that most of the train and validation losses are in the high range of 1.0 and 2.0 .  The training loss in Figure 4 looks bet

In [67]:
# save as text file 

with open('blogsummary.txt', 'w') as f:
    f.write(text)

# Bonus: Make It An Audiobook!

In [73]:
!pip install gtts

In [75]:
from gtts import gTTS 
from IPython.display import Audio 

tts = gTTS(text) 
tts.save('my_audiobook.wav') 
sound_file = 'my_audiobook.wav'
Audio(sound_file, autoplay=True) 